In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
DIVERSITY_UTILITY = 0.25
FRAC_ADMIT = 0.5


###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

df_stratum_utility = df[['R','T','B_p']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['B_p'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
len(df)

100000

In [4]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [5]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','ml_outcomes']].groupby(['R','T']).sum().reset_index()

In [6]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes']].groupby(['R','T']).count().reset_index()
df_count.columns = ['R','T','Count']
df_count['N'] = df_count['Count']

In [7]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [8]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,ml_outcomes,N
0,191,0,0,14,0.00,1
1,186,1,0,15,0.00,2
2,190,2,0,16,0.00,2
4,189,4,0,18,0.00,6
5,188,5,0,19,0.00,10
...,...,...,...,...,...,...
46,4,46,0,60,1088.08,1876
45,3,45,0,59,1100.26,1897
47,2,47,0,61,1114.24,1741
44,1,44,0,58,1124.04,1972


### Setup optimization problem 

In [9]:
from ortools.linear_solver import pywraplp


In [10]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [11]:
dff

,level_0,index,R,T,ml_outcomes,N
0,191,0,0,14,0.00,1
1,186,1,0,15,0.00,2
2,190,2,0,16,0.00,2
3,165,3,0,17,2.00,5
4,189,4,0,18,0.00,6
...,...,...,...,...,...,...
187,160,187,1,95,2.50,2
188,161,188,1,97,2.50,2
189,174,189,1,98,1.25,1
190,172,190,1,99,1.25,1


In [12]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [13]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [14]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [15]:
# Now we have one constraint
solver.NumConstraints()

1

## Add CF Fair constraints

In [16]:
from collections import Counter

def convertListToProb(raw_list):
    counts = dict(Counter(raw_list))
    probs = {}
    for test_score in counts:
        probs[test_score]  = counts[test_score]/float(len(raw_list))
    return [(probs[t], t) for t in probs]

In [17]:
T_blacks_list = df[df['R']==0][['T','T_black_star']].groupby('T')['T_black_star'].apply(list).reset_index(name='T_blacks')


In [18]:
T_blacks_list['probs'] = T_blacks_list['T_blacks'].apply(convertListToProb)


In [19]:
T_blacks_list

,T,T_blacks,probs
0,14,[15],"[(1.0, 15)]"
1,15,"[12, 16]","[(0.5, 12), (0.5, 16)]"
2,16,"[16, 15]","[(0.5, 16), (0.5, 15)]"
3,17,"[17, 16, 15, 16, 15]","[(0.2, 17), (0.4, 16), (0.4, 15)]"
4,18,"[17, 17, 17, 18, 19, 17]","[(0.6666666666666666, 17), (0.1666666666666666..."
...,...,...,...
100,114,"[105, 104]","[(0.5, 105), (0.5, 104)]"
101,115,"[107, 105]","[(0.5, 107), (0.5, 105)]"
102,116,[107],"[(1.0, 107)]"
103,120,[111],"[(1.0, 111)]"


In [20]:
didntexist = 0
exists = 0
for ix, row in T_blacks_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    White_T = row['T']
    Blacks_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(0.0, White_T)], -1.0)
    for prob in Blacks_Ts:
        if (1.0, prob[1]) not in vars_cache:
            vars_cache[(1.0, prob[1])] = solver.NumVar(0.0, 1.0, str((1.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
        cf_fair_stratum.SetCoefficient(vars_cache[(1.0, prob[1])], prob[0])
    

In [21]:
didntexist,exists

(10, 547)

In [22]:
T_whites_list = df[df['R']==1][['T','T_white_star']].groupby('T')['T_white_star'].apply(list).reset_index(name='T_whites')

In [23]:
T_whites_list['probs'] = T_whites_list['T_whites'].apply(convertListToProb)


In [24]:
didntexist = 0
exists = 0

for ix, row in T_whites_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    Black_T = row['T']
    White_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(1.0, Black_T)], -1.0)
    for prob in White_Ts:
        if (0.0, prob[1]) not in vars_cache:
            vars_cache[(0.0, prob[1])] = solver.NumVar(0.0, 1.0, str((0.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
            
        cf_fair_stratum.SetCoefficient(vars_cache[(0.0, prob[1])], prob[0])
    

In [25]:
cf_fair_stratum.basis_status

<bound method Constraint.basis_status of <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7ff77085edb0> >>

In [26]:
solver.NumConstraints()

193

## Solve linear program

In [27]:
solver.ABNORMAL

4

In [28]:
status = solver.Solve()


In [29]:
status

0

In [30]:
solver.OPTIMAL

0

In [31]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [32]:
df_decisions

,row_id,decision
0,0,0.499361
1,1,0.499361
2,2,0.499361
3,3,0.499361
4,4,0.499361
...,...,...
187,187,0.981751
188,188,1.000000
189,189,1.000000
190,190,1.000000


In [33]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
43,0,43,0,57,1129.52,2017,43,0.499361
44,1,44,0,58,1124.04,1972,44,0.499361
47,2,47,0,61,1114.24,1741,47,0.499361
45,3,45,0,59,1100.26,1897,45,0.499361
46,4,46,0,60,1088.08,1876,46,0.499361
...,...,...,...,...,...,...,...,...
6,187,6,0,20,0.00,11,6,0.499361
5,188,5,0,19,0.00,10,5,0.499361
4,189,4,0,18,0.00,6,4,0.499361
2,190,2,0,16,0.00,2,2,0.499361


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
43,0,43,0,57,1129.52,2017,43,0.499361
44,1,44,0,58,1124.04,1972,44,0.499361
47,2,47,0,61,1114.24,1741,47,0.499361
45,3,45,0,59,1100.26,1897,45,0.499361
46,4,46,0,60,1088.08,1876,46,0.499361
...,...,...,...,...,...,...,...,...
6,187,6,0,20,0.00,11,6,0.499361
5,188,5,0,19,0.00,10,5,0.499361
4,189,4,0,18,0.00,6,4,0.499361
2,190,2,0,16,0.00,2,2,0.499361


In [35]:
xxx.sort_values(by='decision',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
104,179,104,0,127,1.00,1,104,1.000000
100,168,100,0,114,2.00,2,100,1.000000
94,149,94,0,108,6.00,6,94,1.000000
95,154,95,0,109,4.00,4,95,1.000000
190,172,190,1,99,1.25,1,190,1.000000
...,...,...,...,...,...,...,...,...
152,51,152,1,59,477.95,605,152,0.499361
149,42,149,1,56,550.42,754,149,0.499361
148,35,148,1,55,616.32,856,148,0.499361
44,1,44,0,58,1124.04,1972,44,0.499361


In [36]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)

In [37]:
admit_decisions = df.merge(xxx,how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_BLACK_POLICY = (admit_decisions['R_y'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R_y'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()


In [38]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Path-Specific Fairness',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()


In [39]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.5000000000000003